<a href="https://colab.research.google.com/github/bjcabral/Coursera_Capstone/blob/master/Latest-06112019_Coursera_Battle_of_the_Neighborhoods_(Week_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import data and create datframes:
We will be working with two datasets, one from Foursquare and one from the West Virginia Department of Transportation.

#### Dataset description
The Foursquare dataset corresponds to all businesses in a 10-mile radious of the Falling Water's WV town. From this dataset we are looking to learn the number and type of businesses in the area, their popularity, and customer's population, as well as the business proximity to potential locations. From the dataset we will also learn if the identified business are competitors or business that could attract potential fast food customers.

The second dataset contains traffic survey for the areas, specifically, traffic volume by the hours for highway exits near potential locations. We hope to learn the traffic patterns and volume to consider potential customers volume other than local residents.

The following section pulls the datasets and create the corresponding dataframes

In [0]:
# Install necessary libraries 
#!conda install -c conda-forge folium=0.5.0 --yes
!pip install -U folium
print('Folium installed')
#!conda install -c conda-forge geopy --yes
!pip install -U geopy
print('geopy library install')


     |████████████████████████████████| 92kB 5.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.9.1 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


Folium installed
     |████████████████████████████████| 102kB 4.5MB/s 
  Found existing installation: geopy 1.17.0
    Uninstalling geopy-1.17.0:
      Successfully uninstalled geopy-1.17.0


geopy library install


In [47]:
# Import necessary libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import folium # Map plotting library
# import clustering libraries
from sklearn.cluster import KMeans

# import plotting library
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

# Kmean libraries for clustering
from sklearn.cluster import KMeans

print('Libraries imported')

Libraries imported


In [48]:
#Set up Foursquare credentials
CLIENT_ID = '3DAUBXTARLYYR4GT30CHA5Z5WZQZMUXL54WPZFFLRTFK34CH' # your Foursquare ID
CLIENT_SECRET = 'FC2KX2BX4WZHA4XZDRQ14HKORJNGDGLYWJ0NKC20R2VKXNKP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 800
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3DAUBXTARLYYR4GT30CHA5Z5WZQZMUXL54WPZFFLRTFK34CH
CLIENT_SECRET:FC2KX2BX4WZHA4XZDRQ14HKORJNGDGLYWJ0NKC20R2VKXNKP


Set up variables to be used in the Foursquare search request URL

The following section sets up the Foursquare URL to search for all business locations in a 10-miles radius. Something to notice here is that we are not searching by search_queary which queries the name of the business, but by category. For this, I had to include the variable &category to the URL and populate the category value to the category for "Fast Food" as found in the Foursquare website.

## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&category=`**CATEGORY**`&radius=`**RADIUS**`&limit=`**LIMIT**


In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [0]:
# Let's get venues for each city. Later we will use each city as a cluster to compare and contrast venues between cities. This will help determing which city will be best location. 
falling_waters = [39.56375, -77.887494]
williamsport = [39.59746, -77.81767]
funkstown = [39.608424, -77.707934]
Halfway = [39.616295, -77.766957]
hagerstown = [39.640182, -77.722691]
clearspring = [39.6562, -77.9317]
hedgesville = [39.55443, -77.994483]
martinsburg = [39.457769, -77.978212]
sharpsburg = [39.457541, -77.749448]
sheperdstown = [39.431799, -77.804895]

city_coordinates = [falling_waters, williamsport, funkstown, Halfway, hagerstown, clearspring, hedgesville, martinsburg, sharpsburg, sheperdstown]


In [58]:
# I am having problems getting all the businesses in the queries above. I can see that there are businesses I know of that are not showing. I will try to explore each city independently to see if more businesses show. 
# looping through the list of coordinates above and adding to a super json file.
import json
cities = []
results = {}
output = {}
for i in range(len(city_coordinates)):
   #print("Processing coordinates {}".format(i))
   #for j in range(len(city_coordinates[i])):
  search_query = ''                 
  #category = '4bf58dd8d48988d16e941735'
  category = ''
  radius = 2000
  limit=200
  latitude = city_coordinates[i][0]
  longitude = city_coordinates[i][1]
  #print(search_query + ' .... OK!')
  url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&category={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, category, radius, LIMIT)
  results = requests.get(url).json()
  fileName = 'jsonData' + str(i) + '.json'
  print(fileName)
  with open(fileName, "w") as outData:
    json.dump(results, outData)
  
  

jsonData0.json
jsonData1.json
jsonData2.json
jsonData3.json
jsonData4.json
jsonData5.json
jsonData6.json
jsonData7.json
jsonData8.json
jsonData9.json


In [0]:
# Read json file into a Pandas dataframe
import json
from pprint import pprint

df_data = cities_venues

# This loop will process all json files and append to a single dataframe
for file_num in range(len(city_coordinates)):
  file_name = 'jsonData' + str(file_num) + '.json'
  with open(file_name, 'r', encoding='utf-8') as data_file:    
    data = json.load(data_file)
    #print("File {} open for reading".format(file_name))
 
  # Create dataframe from json data
  # Create dataframe from json data
  venues = data['response']['groups'][0]['items']
  #print("Normalizing file" + file_name)
  cities_venues_in = json_normalize(venues) # flatten JSON

  # filter columns
  #print("Filtering data for file {}".format(file_name))
  filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.distance', 'venue.location.address', 'venue.location.city']
  cities_venues_in = cities_venues_in.loc[:, filtered_columns]

  # filter the category for each row
  #print("Getting venues categories for file {}".format(file_name))
  cities_venues_in['venue.categories'] = cities_venues_in.apply(get_category_type, axis=1)

  # clean columns
  #print("Cleaning columns for file {}".format(file_name))
  cities_venues_in.columns = [col.split(".")[-1] for col in cities_venues_in.columns]
  #print("Appending file {} to cities_venues_out".format(file_name))
  cities_venues_df = df_data.append(cities_venues_in, ignore_index = True, sort=False)
  
  


In [0]:
# Display dataframe
# cities_venues_df

In [85]:
# Drop data from dataframe for which City equals NaN
cities_venues_df.dropna(subset=['city'], inplace=True)
cities_venues_df
#Print one city only
#cities_venues_df[cities_venues_df['city'] == "Sharpsburg"]

,name,categories,lat,lng,distance,address,city
39,Antietam National Battlefield,Historic Site,39.473969,-77.744836,16123,5831 Dunker Church Rd,Sharpsburg
44,Antietam National Battlefield Park Visitor's C...,History Museum,39.473924,-77.744997,16117,Antietam National Battlefield,Sharpsburg
94,The Cornfield,Historic Site,39.480930,-77.747427,15422,Antietam National Battlefield Park,Sharpsburg


In [82]:
# Map all the business locations for each city, cities are indicated by colored circles.
cities_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Falling Waters

# add a red circle marker to represents each town
folium.CircleMarker(
    location=falling_waters,
    radius=50,
    color='red',
    popup='Falling Waters, WV',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'red',
    parseHTML=False).add_to(cities_map)

# add a red circle marker to represent town
folium.CircleMarker(
    location=williamsport,
    radius=50,
    color='yellow',
    popup='Williamsport, MD',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'yellow',
    parseHTML=False).add_to(cities_map)

# add a red circle marker to represent town
folium.CircleMarker(
    location=funkstown,
    radius=50,
    color='green',
    popup='Funkstown, MD',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'green',
    parseHTML=False).add_to(cities_map)

# add a red circle marker to represent town
folium.CircleMarker(
    location=Halfway,
    radius=50,
    color='purple',
    popup='Halfway, MD',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'purple',
    parseHTML=False).add_to(cities_map)

# add a red circle marker to represents town
folium.CircleMarker(
    location=hagerstown,
    radius=50,
    color='orange',
    popup='Hagerstown, MD',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'orange',
    parseHTML=False).add_to(cities_map)

# add a red circle marker to represents town
folium.CircleMarker(
    location=hagerstown,
    radius=50,
    color='orange',
    popup='Hagerstown, MD',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'orange',
    parseHTML=False).add_to(cities_map)

# add a red circle marker to represents town
folium.CircleMarker(
    location=clearspring,
    radius=50,
    color='black',
    popup='Clear Spring, MD',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'black',
    parseHTML=False).add_to(cities_map)

# add a red circle marker to represents town
folium.CircleMarker(
    location=hedgesville,
    radius=50,
    color='blue',
    popup='Hedgesville, WV',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'blue',
    parseHTML=False).add_to(cities_map)

# add a red circle marker to represents town
folium.CircleMarker(
    location=martinsburg,
    radius=50,
    color='cyan',
    popup='Martinsburg, WV',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'cyan',
    parseHTML=False).add_to(cities_map)

# add a red circle marker to represents town
folium.CircleMarker(
    location=sharpsburg,
    radius=50,
    color='tan',
    popup='Sharpsburg, MD',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'tan',
    parseHTML=False).add_to(cities_map)

# add a red circle marker to represents town
folium.CircleMarker(
    location=sheperdstown,
    radius=50,
    color='light blue',
    popup='Sheperdstown, WV',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'light blue',
    parseHTML=False).add_to(cities_map)

In [0]:
 # Businesses are displayed as blue circles and cities as big colored circles. Click on each to display information including distance from proposed location in big Red Circle
for lat, lng, label, distance in zip(cities_venues_df['lat'], 
                                     cities_venues_df['lng'], cities_venues_df['categories'], cities_venues_df['distance']):
    miles = round(distance/1610, 2)
    popup_label = str(label) + "  " + str(miles) + " miles."
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        color='blue',
        popup=popup_label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6,
        parseHTML=False).add_to(cities_map)

In [84]:
# display map
cities_map

In [66]:
# Identify Unique categories of businesses including restaurants in the area
print("There are {} unique categories.".format(len(cities_venues_df['categories'].unique())))

There are 69 unique categories.


In [86]:
# Analyze nearby businesses
# one hot encoding
# one hot encoding
cities_onehot = pd.get_dummies(cities_venues_df[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cities_onehot['city'] = cities_venues_df['city'] 

# move neighborhood column to the first column
fixed_columns = [cities_onehot.columns[-1]] + list(cities_onehot.columns[:-1])
cities_onehot = cities_onehot[fixed_columns]

cities_onehot

,city,American Restaurant,Antique Shop,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bar,Bookstore,Breakfast Spot,Brewery,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Department Store,Diner,Discount Store,Donut Shop,Fabric Shop,Farmers Market,Fast Food Restaurant,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Government Building,Gym,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Lingerie Store,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Movie Theater,New American Restaurant,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Restaurant,Roller Rink,Sandwich Place,Seafood Restaurant,Shop & Service,Sporting Goods Shop,Steakhouse,Supermarket,Tea Room,Thai Restaurant,Tourist Information Center,Trail,Vegetarian / Vegan Restaurant,Wine Shop
0,Falling Waters,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,Williamsport,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Williamsport,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Williamsport,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Williamsport,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Falling Waters,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Williamsport,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Williamsport,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Hagerstown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Halfway,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
# Take the mean of the frequency of occurence for each business category
cities_grouped = cities_onehot.groupby('city').mean().reset_index()
cities_grouped

,city,American Restaurant,Art Museum,BBQ Joint,Bakery,Bar,Bookstore,Breakfast Spot,Brewery,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Department Store,Diner,Donut Shop,Fabric Shop,Fast Food Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,German Restaurant,Government Building,Gym,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Lingerie Store,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Movie Theater,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Restaurant,Roller Rink,Sandwich Place,Seafood Restaurant,Shop & Service,Sporting Goods Shop,Steakhouse,Supermarket,Thai Restaurant,Trail,Vegetarian / Vegan Restaurant
0,Clear Spring,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000
1,Falling Waters,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000,0.000000
2,Funkstown,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000
3,Hagerstown,0.092308,0.015385,0.015385,0.015385,0.015385,0.030769,0.000000,0.000,0.015385,0.015385,0.076923,0.046154,0.030769,0.030769,0.000000,0.015385,0.015385,0.030769,0.015385,0.015385,0.015385,0.015385,0.0,0.000000,0.000000,0.000000,0.076923,0.030769,0.015385,0.015385,0.030769,0.015385,0.000,0.015385,0.015385,0.030769,0.015385,0.015385,0.030769,0.015385,0.000000,0.030769,0.0,0.015385,0.030769,0.015385,0.000000,0.015385,0.030769,0.0,0.015385,0.000,0.000000
4,Halfway,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000
5,Hedgesville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000
6,Martinsburg,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.066667,0.000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.133333,0.000000,0.000000,0.066667,0.000000,0.0,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.0,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.0,0.000000,0.000,0.06666

In [0]:
# Most common venues in the area
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [88]:
num_top_venues = 10

for count in cities_grouped['city']:
    print("----"+count+"----")
    temp = cities_grouped[cities_grouped['city'] == count].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Clear Spring----
                      venue  freq
0                     Diner   1.0
1       American Restaurant   0.0
2        Italian Restaurant   0.0
3       Japanese Restaurant   0.0
4            Lingerie Store   0.0
5              Liquor Store   0.0
6                    Lounge   0.0
7  Mediterranean Restaurant   0.0
8        Mexican Restaurant   0.0
9             Movie Theater   0.0


----Falling Waters----
                      venue  freq
0               Supermarket   0.5
1               Pizza Place   0.5
2       American Restaurant   0.0
3                 Pet Store   0.0
4        Italian Restaurant   0.0
5       Japanese Restaurant   0.0
6            Lingerie Store   0.0
7              Liquor Store   0.0
8                    Lounge   0.0
9  Mediterranean Restaurant   0.0


----Funkstown----
                      venue  freq
0                       Bar   1.0
1       American Restaurant   0.0
2                  Pharmacy   0.0
3        Italian Restaurant   0.0
4       Japanese

In [89]:
# Convert into dataframe
# Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
  
  
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cities_venues_sorted = pd.DataFrame(columns=columns)
cities_venues_sorted['city'] = cities_grouped['city']

for ind in np.arange(cities_grouped.shape[0]):
    cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cities_grouped.iloc[ind, :], num_top_venues)

cities_venues_sorted

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Clear Spring,Diner,Vegetarian / Vegan Restaurant,History Museum,Gym,Government Building,German Restaurant,Gas Station,Furniture / Home Store,Frozen Yogurt Shop,Fast Food Restaurant
1,Falling Waters,Supermarket,Pizza Place,Vegetarian / Vegan Restaurant,Cosmetics Shop,Government Building,German Restaurant,Gas Station,Furniture / Home Store,Frozen Yogurt Shop,Fast Food Restaurant
2,Funkstown,Bar,Vegetarian / Vegan Restaurant,Department Store,Gym,Government Building,German Restaurant,Gas Station,Furniture / Home Store,Frozen Yogurt Shop,Fast Food Restaurant
3,Hagerstown,American Restaurant,Clothing Store,Hotel,Coffee Shop,Park,Cosmetics Shop,Department Store,Fast Food Restaurant,Steakhouse,Bookstore
4,Halfway,Furniture / Home Store,Restaurant,Vegetarian / Vegan Restaurant,Cosmetics Shop,Gym,Government Building,German Restaurant,Gas Station,Frozen Yogurt Shop,Fast Food Restaurant
5,Hedgesville,Italian Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Cosmetics Shop,Government Building,German Restaurant,Gas Station,Furniture / Home Store,Frozen Yogurt Shop,Fast Food Restaurant
6,Martinsburg,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Gas Station,Italian Restaurant,Diner,Mexican Restaurant,Chinese Restaurant,Pharmacy,Gym
7,Sharpsburg,Historic Site,History Museum,Gym,Government Building,German Restaurant,Gas Station,Furniture / Home Store,Frozen Yogurt Shop,Fast Food Restaurant,Fabric Shop
8,Shepherdstown,Government Building,Vegetarian / Vegan Restaurant,History Museum,Gym,German Restaurant,Gas Station,Furniture / Home Store,Frozen Yogurt Shop,Fast Food Restaurant,Fabric Shop
9,Williamsport,Bar,Trail,Gas Station,Brewery,Café,Pizza Place,Liquor Store,Vegetarian / Vegan Restaurant,Diner,Government Building


In [90]:
# Businesses count by city
cities_venues_df.groupby('city').count()

,name,categories,lat,lng,distance,address
city,,,,,,
Clear Spring,1,1,1,1,1,1
Falling Waters,2,2,2,2,2,2
Funkstown,1,1,1,1,1,1
Hagerstown,65,65,65,65,65,64
Halfway,2,2,2,2,2,0
Hedgesville,2,2,2,2,2,2
Martinsburg,15,15,15,15,15,12
Sharpsburg,3,3,3,3,3,3
Shepherdstown,44,44,44,44,44,41


In [93]:
# Use cities_grouped information for clustering, after droping the citi column
cities_grouped_clustering = cities_grouped.drop('city', 1)

kclusters = 10
# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cities_grouped_clustering)
# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([5, 7, 2, 9, 6, 0, 1, 4, 3, 8], dtype=int32)

In [95]:
# Create dataframe with clusters and most popular venues
# add clustering labels
cities_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
# Merge original dataframe with the dataframe including the labels
cities_merged = cities_venues_df
cities_merged = cities_merged.join(cities_venues_sorted.set_index('city'), on='city')

cities_merged.head()

,name,categories,lat,lng,distance,address,city,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Food Lion Grocery Store,Supermarket,39.577089,-77.876325,103,78 Worchester Dr,Falling Waters,7,Supermarket,Pizza Place,Vegetarian / Vegan Restaurant,Cosmetics Shop,Government Building,German Restaurant,Gas Station,Furniture / Home Store,Frozen Yogurt Shop,Fast Food Restaurant
1,Cushwa Brewing Company,Brewery,39.594582,-77.806529,6396,10212 Governor Lane Blvd Unit 1012,Williamsport,8,Bar,Trail,Gas Station,Brewery,Café,Pizza Place,Liquor Store,Vegetarian / Vegan Restaurant,Diner,Government Building
2,C&O Canal National Historical Park,Trail,39.600769,-77.826574,5119,205 W Potomac St,Williamsport,8,Bar,Trail,Gas Station,Brewery,Café,Pizza Place,Liquor Store,Vegetarian / Vegan Restaurant,Diner,Government Building
3,Tony's Pizza,Pizza Place,39.598960,-77.821000,5437,10 E Salisbury St,Williamsport,8,Bar,Trail,Gas Station,Brewery,Café,Pizza Place,Liquor Store,Vegetarian / Vegan Restaurant,Diner,Government Building
4,Redmen Tribe #84,Bar,39.578856,-77.808019,5953,16129 Lappans Rd,Williamsport,8,Bar,Trail,Gas Station,Brewery,Café,Pizza Place,Liquor Store,Vegetarian / Vegan Restaurant,Diner,Government Building


In [99]:
# Overlay the colored cities to see how accurate the cluster is in relation to the city coordinates as we did in the prior map. Of course, this can be also seen using the name of the cities. But as you noticed below, the algorighm merged
# some of the cities, for example, Halfway and Hagerstown are all under cluster 9, while Funkstown has its own cluster 2
# Map all the business locations for each city, cities are indicated by colored circles.

# Visualize the cluster in a map
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add a red circle marker to represents each town
folium.CircleMarker(
    location=falling_waters,
    radius=50,
    color='red',
    popup='Falling Waters, WV',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'red',
    parseHTML=False).add_to(map_clusters)

# add a red circle marker to represent town
folium.CircleMarker(
    location=williamsport,
    radius=50,
    color='yellow',
    popup='Williamsport, MD',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'yellow',
    parseHTML=False).add_to(map_clusters)

# add a red circle marker to represent town
folium.CircleMarker(
    location=funkstown,
    radius=50,
    color='green',
    popup='Funkstown, MD',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'green',
    parseHTML=False).add_to(map_clusters)

# add a red circle marker to represent town
folium.CircleMarker(
    location=Halfway,
    radius=50,
    color='purple',
    popup='Halfway, MD',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'purple',
    parseHTML=False).add_to(map_clusters)

# add a red circle marker to represents town
folium.CircleMarker(
    location=hagerstown,
    radius=50,
    color='orange',
    popup='Hagerstown, MD',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'orange',
    parseHTML=False).add_to(map_clusters)

# add a red circle marker to represents town
folium.CircleMarker(
    location=hagerstown,
    radius=50,
    color='orange',
    popup='Hagerstown, MD',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'orange',
    parseHTML=False).add_to(map_clusters)

# add a red circle marker to represents town
folium.CircleMarker(
    location=clearspring,
    radius=50,
    color='black',
    popup='Clear Spring, MD',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'black',
    parseHTML=False).add_to(map_clusters)

# add a red circle marker to represents town
folium.CircleMarker(
    location=hedgesville,
    radius=50,
    color='blue',
    popup='Hedgesville, WV',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'blue',
    parseHTML=False).add_to(map_clusters)

# add a red circle marker to represents town
folium.CircleMarker(
    location=martinsburg,
    radius=50,
    color='cyan',
    popup='Martinsburg, WV',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'cyan',
    parseHTML=False).add_to(map_clusters)

# add a red circle marker to represents town
folium.CircleMarker(
    location=sharpsburg,
    radius=50,
    color='tan',
    popup='Sharpsburg, MD',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'tan',
    parseHTML=False).add_to(map_clusters)

# add a red circle marker to represents town
folium.CircleMarker(
    location=sheperdstown,
    radius=50,
    color='light blue',
    popup='Sheperdstown, WV',
    fill = True,
    fill_opacity = 0.6,
    fill_color = 'light blue',
    parseHTML=False).add_to(map_clusters)


# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cities_merged['lat'], cities_merged['lng'], cities_merged['city'], cities_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [101]:
# Use Standar Scaler
from sklearn.preprocessing import StandardScaler
X = cities_grouped_clustering.values[:,1:]
X = np.nan_to_num(X)
cluster_data = StandardScaler().fit_transform(X)
cluster_data



array([[-0.33333333, -0.40985242, -0.33333333, -0.42118299, -0.33333333,
        -0.33333333, -0.33333333, -0.37666415, -0.40985242, -0.33333333,
        -0.33333333, -0.33333333, -0.33333333,  2.99333925, -0.33333333,
        -0.33333333, -0.40985242, -0.33333333, -0.34460626, -0.5173194 ,
        -0.33333333, -0.33333333, -0.33333333, -0.33333333, -0.33333333,
        -0.33333333, -0.33333333, -0.33333333, -0.3917049 , -0.33333333,
        -0.33333333, -0.33333333, -0.33333333, -0.33333333, -0.46231584,
        -0.33333333, -0.33333333, -0.33333333, -0.33333333, -0.33333333,
        -0.63416053, -0.33333333, -0.33333333, -0.33333333, -0.33333333,
        -0.33333333, -0.33333333, -0.46231584, -0.33333333, -0.33333333,
        -0.33333333, -0.33333333],
       [-0.33333333, -0.40985242, -0.33333333, -0.42118299, -0.33333333,
        -0.33333333, -0.33333333, -0.37666415, -0.40985242, -0.33333333,
        -0.33333333, -0.33333333, -0.33333333, -0.35741364, -0.33333333,
        -0.33333

In [109]:
# Model into 10 clusters representing each city. 
num_clusters = 10
k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_data)
labels = k_means.labels_

print(labels)

[8 5 1 0 6 7 3 4 9 2]


In [110]:
# Check the centroid values by averaging the clusters
cities_grouped_clustering['Labels'] = labels
cities_grouped_clustering

,American Restaurant,Art Museum,BBQ Joint,Bakery,Bar,Bookstore,Breakfast Spot,Brewery,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Department Store,Diner,Donut Shop,Fabric Shop,Fast Food Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,German Restaurant,Government Building,Gym,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Lingerie Store,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Movie Theater,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Restaurant,Roller Rink,Sandwich Place,Seafood Restaurant,Shop & Service,Sporting Goods Shop,Steakhouse,Supermarket,Thai Restaurant,Trail,Vegetarian / Vegan Restaurant,Labels
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,8
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000,0.000000,5
2,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,1
3,0.092308,0.015385,0.015385,0.015385,0.015385,0.030769,0.000000,0.000,0.015385,0.015385,0.076923,0.046154,0.030769,0.030769,0.000000,0.015385,0.015385,0.030769,0.015385,0.015385,0.015385,0.015385,0.0,0.000000,0.000000,0.000000,0.076923,0.030769,0.015385,0.015385,0.030769,0.015385,0.000,0.015385,0.015385,0.030769,0.015385,0.015385,0.030769,0.015385,0.000000,0.030769,0.0,0.015385,0.030769,0.015385,0.000000,0.015385,0.030769,0.0,0.015385,0.000,0.000000,0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,6
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,7
6,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.066667,0.000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.133333,0.000000,0.000000,0.066667,0.000000,0.0,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.0,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.0,0.000000,0.000,0.066667,3
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [111]:
# Centroid values
# The algorithms generated 10 groups, this business groups are similar based on their characteristics in the data. We can use this data to compare a group against the others and determine which groups are similar and could compete with our
# proposed location which in this case is group 7, Falling Waters, WV. And the new venue is in the Fast Food category
cities_grouped_clustering.groupby('Labels').mean()

,American Restaurant,Art Museum,BBQ Joint,Bakery,Bar,Bookstore,Breakfast Spot,Brewery,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Department Store,Diner,Donut Shop,Fabric Shop,Fast Food Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,German Restaurant,Government Building,Gym,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Lingerie Store,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Movie Theater,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Restaurant,Roller Rink,Sandwich Place,Seafood Restaurant,Shop & Service,Sporting Goods Shop,Steakhouse,Supermarket,Thai Restaurant,Trail,Vegetarian / Vegan Restaurant
Labels,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.092308,0.015385,0.015385,0.015385,0.015385,0.030769,0.000000,0.000,0.015385,0.015385,0.076923,0.046154,0.030769,0.030769,0.000000,0.015385,0.015385,0.030769,0.015385,0.015385,0.015385,0.015385,0.0,0.000000,0.000000,0.000000,0.076923,0.030769,0.015385,0.015385,0.030769,0.015385,0.000,0.015385,0.015385,0.030769,0.015385,0.015385,0.030769,0.015385,0.000000,0.030769,0.0,0.015385,0.030769,0.015385,0.000000,0.015385,0.030769,0.0,0.015385,0.000,0.000000
1,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.125,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.125,0.000000
3,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.066667,0.000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.133333,0.000000,0.000000,0.066667,0.000000,0.0,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.0,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.0,0.000000,0.000,0.066667
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.666667,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000
7,0.000000,0.000000,0.00